In [1]:
!pip install pyodbc pandas geopy requests sqlalchemy psycopg2-binary tqdm


     -------------------------------------- 125.4/125.4 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import pyodbc
import requests
from geopy.geocoders import Nominatim
import math
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from time import sleep
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [2]:
# Definir a classe Config e Server para armazenar as configurações do banco de dados
class Config:
    DATABASE_USERNAME_ATF = 'seu_usuario'  # Substitua pelo seu usuário
    DATABASE_PASSWORD_ATF = 'sua_senha'    # Substitua pela sua senha
    DSN_NAME = 'Informix'                  # Substitua pelo nome do DSN que você configurou

class Server:
    app = Config()

server = Server()

# Função para obter a conexão com o banco de dados
def get_atf_engine() -> pyodbc.Connection:
    username = f'{server.app.DATABASE_USERNAME_ATF}'
    password = f'{server.app.DATABASE_PASSWORD_ATF}'
    dsn_name = f'{server.app.DSN_NAME}'
    return pyodbc.connect(f'DSN={dsn_name};UID={username};PWD={password}')


In [3]:
tbfis_hinfeletronica = pd.read_csv('tbfis_hinfeletron_not_null.csv', sep=',')
tbfis_hinfeletronica

,sqhinfeletron,sqhumanoinst,municipio,sqtpdochuminst,cdpais,nrcep,nrdocumento,nopais,idestrangeiro,dscomplemento,nobairro,nologradouro,noemail,norazaosocial,nrinscricao,nrinscrsuframa,nrresidencia,nrtelefone,sgufhinfeletron,tsatualizacao
0,15569417,11783220,PARNAMIRIM,3,NaN,59152505,10446336467,Brasil,NaN,NaN,NOVA PARNAMIRIM,R UMARIZAL,NaN,JULIA CRISTINA M C DA SILVA,NaN,NaN,69,NaN,RN,2016-09-21 06:12:29.000
1,15569418,11783221,NATAL,3,NaN,59074060,5889831445,Brasil,NaN,NaN,FELIPE CAMARAO,R POTIGUARES,NaN,ROSIANE ROCHA RAFAEL,NaN,NaN,0,NaN,RN,2016-09-21 06:13:23.000
2,15569419,11783222,MACAIBA,3,NaN,59280000,9907707457,Brasil,NaN,NaN,CENTRO,R MIGUEL DA CRUZ,NaN,ALINE DANIELLE MENDES BEZERRA,NaN,NaN,19,NaN,RN,2016-09-21 06:13:25.000
3,15569420,11783223,NATAL,3,NaN,59132690,3363205406,Brasil,NaN,NaN,PAJUCARA,R DOS IMIGRANTES,NaN,AURICELIA MARIA DE ALBURQUERQUE,NaN,NaN,11,NaN,RN,2016-09-21 06:13:27.000
4,15569421,8651723,NATAL,3,NaN,59054590,2150325438,Brasil,NaN,NaN,LAGOA NOVA,AV BERNARDO VIEIRA,NaN,KALIANE KEILLY BRITO R DA SILVA,NaN,NaN,2448,NaN,RN,2016-09-21 06:13:28.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15570577,11783614,SAO PAULO,3,NaN,4854270,40379455846,BRASIL,NaN,NaN,JD ALMEIDA PRADO,STO ANTONIO,NaN,JUCEILMA GREGORIO VALENA DA SILVA,NaN,NaN,16,NaN,SP,2016-09-21 09:28:21.000
996,15570578,511190,CAMPINA GRANDE,3,NaN,58100001,6603521091,BRASIL,NaN,S/N,BODOCONGO,AVENIDA MARECHAL FLORIANO PEIXOTO,NaN,WILLIAM IMPERIANO DE CRISTO,NaN,NaN,S/N,NaN,PB,2016-09-21 09:28:22.000
997,15570581,9430452,JOAO PESSOA,3,NaN,58000000,6663272462,BRASIL,NaN,NaN,ALTO DO MATEUS,RUA JOAO AMERICO DE CARVALHO SN,NaN,NILSON DE ANDRADE QUIRINO,NaN,NaN,SN,NaN,PB,2016-09-21 09:28:37.000
998,15570583,11738183,JUAZEIRINHO,2,NaN,58660000,25460331000135,BRASIL,NaN,NaN,DISTRITO INDUSTRIAL,RUA RES DISTRITO BARRA DE JUAZEIRINHO,NaN,ELEICAO 2016 SANDRA MARIA PAULINO AMARO VEREADOR,NaN,NaN,S/N,NaN,PB,2016-09-21 09:29:00.000


In [4]:
df = tbfis_hinfeletronica.copy()  # Certifique-se de que df seja o DataFrame correto

In [5]:
%%time

# Criação de um endereço completo baseado em várias colunas do DataFrame
df["enderecoCompleto"] = df.apply(lambda row: f"{row['nologradouro']}, {row['nrresidencia']}, {row['municipio']}, {row['sgufhinfeletron']}, BRASIL", axis=1)
enderecos_emitentes = df[["sqhinfeletron","nrcep", "nologradouro", "municipio", "sgufhinfeletron", "enderecoCompleto"]]

# Contagem de endereços completos e ordenação por frequência
enderecos_emitentes['contagemEnderecoCompleto'] = enderecos_emitentes['enderecoCompleto'].map(enderecos_emitentes['enderecoCompleto'].value_counts())
enderecos_emitentes = enderecos_emitentes.sort_values(by='contagemEnderecoCompleto', ascending=False)
enderecos_emitentes.drop_duplicates(subset=['enderecoCompleto'], keep='first', inplace=True)
enderecos_emitentes.reset_index(drop=True, inplace=True)

enderecos_emitentes


CPU times: total: 156 ms
Wall time: 814 ms


<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto
0,15570529,58310000,ROD.BR 230 KM 12,CABEDELO,PB,"ROD.BR 230 KM 12, 0, CABEDELO, PB, BRASIL",2
1,15570221,9715021,RUA AMERICO BRASILIENSE,SAO BERNARDO DO CAMPO,SP,"RUA AMERICO BRASILIENSE, 596, SAO BERNARDO DO ...",2
2,15570315,86790000,AV GRALHA AZUL,LOBATO,PR,"AV GRALHA AZUL, 1196, LOBATO, PR, BRASIL",2
3,15570114,58706310,LIMA CAMPOS,PATOS,PB,"LIMA CAMPOS, 123, PATOS, PB, BRASIL",2
4,15569702,62800000,RUA CORONEL ALEXANDRINO,ARACATI,CE,"RUA CORONEL ALEXANDRINO, 750, ARACATI, CE, BRASIL",2
...,...,...,...,...,...,...,...
984,15569780,58040250,SAO SEBASTIAO,JOAO PESSOA,PB,"SAO SEBASTIAO, 436, JOAO PESSOA, PB, BRASIL",1
985,15569781,58200000,Rua Carlos Gomes,GUARABIRA,PB,"Rua Carlos Gomes, 125, GUARABIRA, PB, BRASIL",1
986,15569782,58338000,Rua Luis de Holanda,PILAR,PB,"Rua Luis de Holanda, 68, PILAR, PB, BRASIL",1
987,15569783,58880000,RUA MANOEL EMIDIO,BREJO DOS SANTOS,PB,"RUA MANOEL EMIDIO, 179, BREJO DOS SANTOS, PB, ...",1


In [6]:
from tqdm import tqdm

# Função para obter coordenadas a partir de um endereço
def get_coordinates_from_address(address):
    geolocator = Nominatim(user_agent="cep_geocoder", timeout=5)
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        return ('NaN', 'NaN')
    except (GeocoderTimedOut, GeocoderServiceError):
        return ('NaN', 'NaN')

# Função para obter coordenadas a partir de um CEP usando a API ViaCEP
def get_coordinates_from_cep(cep, retries=3):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200 and 'erro' not in response.json():
                address = response.json()
                formatted_address = f"{address.get('logradouro', '')}, {address['localidade']}, {address['uf']}, Brasil"
                return get_coordinates_from_address(formatted_address)
            else:
                return ('NaN', 'NaN')
        except requests.exceptions.RequestException:
            sleep(1)
    return ('NaN', 'NaN')

# Aplicação das funções para obter coordenadas no DataFrame
def apply_coordinates_to_dataframe(df, address_column, cep_column):
    latitudes = []
    longitudes = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        address = row[address_column]
        cep = row[cep_column]
        if pd.notna(address):
            lat, lon = get_coordinates_from_address(address)
        elif pd.notna(cep):
            lat, lon = get_coordinates_from_cep(cep)
        else:
            lat, lon = ('NaN', 'NaN')
        latitudes.append(lat)
        longitudes.append(lon)
    df['latitude'] = latitudes
    df['longitude'] = longitudes
    return df

# Aplicando a função no DataFrame de endereços
df = apply_coordinates_to_dataframe(enderecos_emitentes, 'enderecoCompleto', 'nrcep')
df

 21%|██        | 204/989 [03:24<11:06,  1.18it/s]

In [ ]:
# Consulta por um identificador específico

df.query('sqhinfeletron == 15569418')
#df['Latitude'].value_counts().get('NaN', 0)

,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto,latitude,longitude
762,15569418,59074060,R POTIGUARES,NATAL,RN,"R POTIGUARES, 0, NATAL, RN, BRASIL",1,NaN,NaN


In [ ]:
# DataFrame final com coordenadas
coordenadas = df[["sqhinfeletron", "latitude", "longitude"]]
coordenadas

,sqhinfeletron,latitude,longitude
0,15569672,NaN,NaN
1,15569702,NaN,NaN
2,15569718,NaN,NaN
3,15569669,NaN,NaN
4,15570113,NaN,NaN
...,...,...,...
984,15570577,-23.517737,-47.131708
985,15570578,-7.238459,-35.925273
986,15570581,NaN,NaN
987,15570583,NaN,NaN


In [ ]:
# Ordenação dos DataFrames
tbfis_hinfeletronica_completo = pd.merge(tbfis_hinfeletronica, coordenadas, on='sqhinfeletron', how='left', suffixes=('', '_lista_enderecos'))
tbfis_hinfeletronica_completo

,sqhinfeletron,sqhumanoinst,municipio,sqtpdochuminst,cdpais,nrcep,nrdocumento,nopais,idestrangeiro,dscomplemento,...,noemail,norazaosocial,nrinscricao,nrinscrsuframa,nrresidencia,nrtelefone,sgufhinfeletron,tsatualizacao,latitude,longitude
0,15569417,11783220,PARNAMIRIM,3,NaN,59152505,10446336467,Brasil,NaN,NaN,...,NaN,JULIA CRISTINA M C DA SILVA,NaN,NaN,69,NaN,RN,2016-09-21 06:12:29.000,NaN,NaN
1,15569418,11783221,NATAL,3,NaN,59074060,5889831445,Brasil,NaN,NaN,...,NaN,ROSIANE ROCHA RAFAEL,NaN,NaN,0,NaN,RN,2016-09-21 06:13:23.000,NaN,NaN
2,15569419,11783222,MACAIBA,3,NaN,59280000,9907707457,Brasil,NaN,NaN,...,NaN,ALINE DANIELLE MENDES BEZERRA,NaN,NaN,19,NaN,RN,2016-09-21 06:13:25.000,NaN,NaN
3,15569420,11783223,NATAL,3,NaN,59132690,3363205406,Brasil,NaN,NaN,...,NaN,AURICELIA MARIA DE ALBURQUERQUE,NaN,NaN,11,NaN,RN,2016-09-21 06:13:27.000,NaN,NaN
4,15569421,8651723,NATAL,3,NaN,59054590,2150325438,Brasil,NaN,NaN,...,NaN,KALIANE KEILLY BRITO R DA SILVA,NaN,NaN,2448,NaN,RN,2016-09-21 06:13:28.000,-5.810005,-35.21256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15570577,11783614,SAO PAULO,3,NaN,4854270,40379455846,BRASIL,NaN,NaN,...,NaN,JUCEILMA GREGORIO VALENA DA SILVA,NaN,NaN,16,NaN,SP,2016-09-21 09:28:21.000,-23.517737,-47.131708
996,15570578,511190,CAMPINA GRANDE,3,NaN,58100001,6603521091,BRASIL,NaN,S/N,...,NaN,WILLIAM IMPERIANO DE CRISTO,NaN,NaN,S/N,NaN,PB,2016-09-21 09:28:22.000,-7.238459,-35.925273
997,15570581,9430452,JOAO PESSOA,3,NaN,58000000,6663272462,BRASIL,NaN,NaN,...,NaN,NILSON DE ANDRADE QUIRINO,NaN,NaN,SN,NaN,PB,2016-09-21 09:28:37.000,NaN,NaN
998,15570583,11738183,JUAZEIRINHO,2,NaN,58660000,25460331000135,BRASIL,NaN,NaN,...,NaN,ELEICAO 2016 SANDRA MARIA PAULINO AMARO VEREADOR,NaN,NaN,S/N,NaN,PB,2016-09-21 09:29:00.000,NaN,NaN


In [ ]:
tbfis_hinfeletronica_completo.sort_values(by='sqhinfeletron', ignore_index=True)

,sqhinfeletron,sqhumanoinst,municipio,sqtpdochuminst,cdpais,nrcep,nrdocumento,nopais,idestrangeiro,dscomplemento,...,noemail,norazaosocial,nrinscricao,nrinscrsuframa,nrresidencia,nrtelefone,sgufhinfeletron,tsatualizacao,latitude,longitude
0,15569417,11783220,PARNAMIRIM,3,NaN,59152505,10446336467,Brasil,NaN,NaN,...,NaN,JULIA CRISTINA M C DA SILVA,NaN,NaN,69,NaN,RN,2016-09-21 06:12:29.000,NaN,NaN
1,15569418,11783221,NATAL,3,NaN,59074060,5889831445,Brasil,NaN,NaN,...,NaN,ROSIANE ROCHA RAFAEL,NaN,NaN,0,NaN,RN,2016-09-21 06:13:23.000,NaN,NaN
2,15569419,11783222,MACAIBA,3,NaN,59280000,9907707457,Brasil,NaN,NaN,...,NaN,ALINE DANIELLE MENDES BEZERRA,NaN,NaN,19,NaN,RN,2016-09-21 06:13:25.000,NaN,NaN
3,15569420,11783223,NATAL,3,NaN,59132690,3363205406,Brasil,NaN,NaN,...,NaN,AURICELIA MARIA DE ALBURQUERQUE,NaN,NaN,11,NaN,RN,2016-09-21 06:13:27.000,NaN,NaN
4,15569421,8651723,NATAL,3,NaN,59054590,2150325438,Brasil,NaN,NaN,...,NaN,KALIANE KEILLY BRITO R DA SILVA,NaN,NaN,2448,NaN,RN,2016-09-21 06:13:28.000,-5.810005,-35.21256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15570577,11783614,SAO PAULO,3,NaN,4854270,40379455846,BRASIL,NaN,NaN,...,NaN,JUCEILMA GREGORIO VALENA DA SILVA,NaN,NaN,16,NaN,SP,2016-09-21 09:28:21.000,-23.517737,-47.131708
996,15570578,511190,CAMPINA GRANDE,3,NaN,58100001,6603521091,BRASIL,NaN,S/N,...,NaN,WILLIAM IMPERIANO DE CRISTO,NaN,NaN,S/N,NaN,PB,2016-09-21 09:28:22.000,-7.238459,-35.925273
997,15570581,9430452,JOAO PESSOA,3,NaN,58000000,6663272462,BRASIL,NaN,NaN,...,NaN,NILSON DE ANDRADE QUIRINO,NaN,NaN,SN,NaN,PB,2016-09-21 09:28:37.000,NaN,NaN
998,15570583,11738183,JUAZEIRINHO,2,NaN,58660000,25460331000135,BRASIL,NaN,NaN,...,NaN,ELEICAO 2016 SANDRA MARIA PAULINO AMARO VEREADOR,NaN,NaN,S/N,NaN,PB,2016-09-21 09:29:00.000,NaN,NaN


In [ ]:
enderecos_emitentes.sort_values(by='sqhinfeletron', ignore_index=True)

,sqhinfeletron,nrcep,nologradouro,municipio,sgufhinfeletron,enderecoCompleto,contagemEnderecoCompleto,latitude,longitude
0,15569417,59152505,R UMARIZAL,PARNAMIRIM,RN,"R UMARIZAL, 69, PARNAMIRIM, RN, BRASIL",1,NaN,NaN
1,15569418,59074060,R POTIGUARES,NATAL,RN,"R POTIGUARES, 0, NATAL, RN, BRASIL",1,NaN,NaN
2,15569419,59280000,R MIGUEL DA CRUZ,MACAIBA,RN,"R MIGUEL DA CRUZ, 19, MACAIBA, RN, BRASIL",1,NaN,NaN
3,15569420,59132690,R DOS IMIGRANTES,NATAL,RN,"R DOS IMIGRANTES, 11, NATAL, RN, BRASIL",1,NaN,NaN
4,15569421,59054590,AV BERNARDO VIEIRA,NATAL,RN,"AV BERNARDO VIEIRA, 2448, NATAL, RN, BRASIL",1,-5.810005,-35.21256
...,...,...,...,...,...,...,...,...,...
984,15570577,4854270,STO ANTONIO,SAO PAULO,SP,"STO ANTONIO, 16, SAO PAULO, SP, BRASIL",1,-23.517737,-47.131708
985,15570578,58100001,AVENIDA MARECHAL FLORIANO PEIXOTO,CAMPINA GRANDE,PB,"AVENIDA MARECHAL FLORIANO PEIXOTO, S/N, CAMPIN...",1,-7.238459,-35.925273
986,15570581,58000000,RUA JOAO AMERICO DE CARVALHO SN,JOAO PESSOA,PB,"RUA JOAO AMERICO DE CARVALHO SN, SN, JOAO PESS...",1,NaN,NaN
987,15570583,58660000,RUA RES DISTRITO BARRA DE JUAZEIRINHO,JUAZEIRINHO,PB,"RUA RES DISTRITO BARRA DE JUAZEIRINHO, S/N, JU...",1,NaN,NaN


In [3]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError



# Configurações de conexão
host = "ceos"
dbname = "teste_espaciais"
user = "postgres"
password = "postgres"
port = "5432"

# Criando o engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')


In [6]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

# Configurações de conexão
host = "ceos"
dbname = "teste_espaciais"
user = "postgres"
password = "postgres"
port = "5432"

# Criando o engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Consulta SQL que você deseja executar
query = "SELECT * FROM TBFIS_HIFELETRON"  # Substitua pelo nome correto da sua tabela

try:
    # Criando a conexão
    with engine.connect() as connection:
        # Executando a consulta e armazenando o resultado em um DataFrame
        df = pd.read_sql(query, con=connection)
        print(df.head())  # Exibindo as primeiras linhas do DataFrame
except SQLAlchemyError as e:
    print(f"Erro ao executar a consulta: {e}")
except Exception as e:
    print(f"Outro erro ocorreu: {e}")



Erro ao executar a consulta: Not an executable object: 'SELECT * FROM TBFIS_HIFELETRON'


In [7]:
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
    connection.close()
except SQLAlchemyError as e:
    print(f"Erro ao conectar: {e}")


Conexão bem-sucedida!


In [12]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

# Configurações de conexão
host = "ceos"
dbname = "teste_espaciais"
user = "postgres"
password = "postgres"
port = "5432"

# Criando o engine SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Consulta SQL para listar tabelas no PostgreSQL
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
"""

try:
    # Criando a conexão
    with engine.connect() as connection:
        # Executando a consulta e armazenando o resultado em um DataFrame
        df = pd.read_sql(query, con=connection)
        print(df.head())  # Exibindo as primeiras linhas do DataFrame
except SQLAlchemyError as e:
    print(f"Erro ao executar a consulta: {e}")
except Exception as e:
    print(f"Outro erro ocorreu: {e}")


Erro ao executar a consulta: Not an executable object: "\nSELECT table_name\nFROM information_schema.tables\nWHERE table_schema = 'public'\n"
